In [ ]:
!pip install pyspark

     |████████████████████████████████| 204.7MB 66kB/s 
     |████████████████████████████████| 204kB 35.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=52dbb686352c20eb040c720a7075c2d280cc4fc3d346ad5de935c6f520cb30cb
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark


In [ ]:
#Initialize SparkSession and SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [ ]:
#Create a Spark Session
SpSession = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
#Get the Spark Context from Spark Session    
SpContext = SpSession.sparkContext

#............................................................................
##   Working with Data Frames

In [42]:
#Create a data frame from a JSON file
empDf = SpSession.read.json("/content/drive/My Drive/customerData.json")
empDf.show()
empDf.printSchema()

+---+------+------+-----------------+------+
|age|deptid|gender|             name|salary|
+---+------+------+-----------------+------+
| 32|   100|  male|Benjamin Garrison|  3000|
| 40|   200|  male|    Holland Drake|  4500|
| 26|   100|  male|  Burks Velasquez|  2700|
| 51|   100|female|    June Rutledge|  4300|
| 44|   200|  male|    Nielsen Knapp|  6500|
+---+------+------+-----------------+------+

root
 |-- age: string (nullable = true)
 |-- deptid: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: string (nullable = true)



In [43]:
#Do Data Frame queries
empDf.select("name").show()

+-----------------+
|             name|
+-----------------+
|Benjamin Garrison|
|    Holland Drake|
|  Burks Velasquez|
|    June Rutledge|
|    Nielsen Knapp|
+-----------------+



In [44]:
empDf.filter(empDf["age"] == 40).show()
empDf.groupBy("gender").count().show()

+---+------+------+-------------+------+
|age|deptid|gender|         name|salary|
+---+------+------+-------------+------+
| 40|   200|  male|Holland Drake|  4500|
+---+------+------+-------------+------+

+------+-----+
|gender|count|
+------+-----+
|female|    1|
|  male|    4|
+------+-----+



In [48]:
empDf.groupBy("deptid").\
    agg({"salary": "avg", "age": "max"}).show()

+------+------------------+--------+
|deptid|       avg(salary)|max(age)|
+------+------------------+--------+
|   200|            5500.0|      44|
|   100|3333.3333333333335|      51|
+------+------------------+--------+



In [52]:
#create a data frame from a list
deptList = [{'name': 'Sales', 'id': "100"},\
     { 'name':'Engineering','id':"200" }]
deptDf = SpSession.createDataFrame(deptList)
deptDf.show()


/usr/local/lib/python3.6/dist-packages/pyspark/sql/session.py:378: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+---+-----------+
| id|       name|
+---+-----------+
|100|      Sales|
|200|Engineering|
+---+-----------+



In [53]:
deptDf = SpSession.createDataFrame(deptList)
deptDf.show()

/usr/local/lib/python3.6/dist-packages/pyspark/sql/session.py:378: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+---+-----------+
| id|       name|
+---+-----------+
|100|      Sales|
|200|Engineering|
+---+-----------+



In [54]:
#cascading operations
empDf.filter(empDf["age"] >30).join(deptDf, \
        empDf.deptid == deptDf.id).\
        groupBy("deptid").\
        agg({"salary": "avg", "age": "max"}).show()

+------+-----------+--------+
|deptid|avg(salary)|max(age)|
+------+-----------+--------+
|   200|     5500.0|      44|
|   100|     3650.0|      51|
+------+-----------+--------+



##   Creating data frames from RDD


In [55]:
from pyspark.sql import Row
lines = SpContext.textFile("/content/drive/My Drive/auto-data.csv")
#remove the first line
datalines = lines.filter(lambda x: "FUELTYPE" not in x)
datalines.count()



197

In [56]:
parts = datalines.map(lambda l: l.split(","))
autoMap = parts.map(lambda p: Row(make=p[0],\
         body=p[4], hp=int(p[7])))
# Infer the schema, and register the DataFrame as a table.
autoDf = SpSession.createDataFrame(autoMap)
autoDf.show()

+----------+---------+---+
|      make|     body| hp|
+----------+---------+---+
|    subaru|hatchback| 69|
| chevrolet|hatchback| 48|
|     mazda|hatchback| 68|
|    toyota|hatchback| 62|
|mitsubishi|hatchback| 68|
|     honda|hatchback| 60|
|    nissan|    sedan| 69|
|     dodge|hatchback| 68|
|  plymouth|hatchback| 68|
|     mazda|hatchback| 68|
|mitsubishi|hatchback| 68|
|     dodge|hatchback| 68|
|  plymouth|hatchback| 68|
| chevrolet|hatchback| 70|
|    toyota|hatchback| 62|
|     dodge|hatchback| 68|
|     honda|hatchback| 58|
|    toyota|hatchback| 62|
|     honda|hatchback| 76|
| chevrolet|    sedan| 70|
+----------+---------+---+
only showing top 20 rows



##   Creating data frames directly from CSV


In [57]:
autoDf1 = SpSession.read.csv("/content/drive/My Drive/auto-data.csv",header=True)
autoDf1.show()

+----------+--------+------+-----+---------+-----+---------+---+----+--------+-------+-----+
|      MAKE|FUELTYPE|ASPIRE|DOORS|     BODY|DRIVE|CYLINDERS| HP| RPM|MPG-CITY|MPG-HWY|PRICE|
+----------+--------+------+-----+---------+-----+---------+---+----+--------+-------+-----+
|    subaru|     gas|   std|  two|hatchback|  fwd|     four| 69|4900|      31|     36| 5118|
| chevrolet|     gas|   std|  two|hatchback|  fwd|    three| 48|5100|      47|     53| 5151|
|     mazda|     gas|   std|  two|hatchback|  fwd|     four| 68|5000|      30|     31| 5195|
|    toyota|     gas|   std|  two|hatchback|  fwd|     four| 62|4800|      35|     39| 5348|
|mitsubishi|     gas|   std|  two|hatchback|  fwd|     four| 68|5500|      37|     41| 5389|
|     honda|     gas|   std|  two|hatchback|  fwd|     four| 60|5500|      38|     42| 5399|
|    nissan|     gas|   std|  two|    sedan|  fwd|     four| 69|5200|      31|     37| 5499|
|     dodge|     gas|   std|  two|hatchback|  fwd|     four| 68|5500| 

##   Creating and working with Temp Tables


In [58]:
autoDf.createOrReplaceTempView("autos")
SpSession.sql("select * from autos where hp > 200").show()

#register a data frame as table and run SQL statements against it
empDf.createOrReplaceTempView("employees")
SpSession.sql("select * from employees where salary > 4000").show()

+-------+-----------+---+
|   make|       body| hp|
+-------+-----------+---+
|porsche|    hardtop|207|
|porsche|    hardtop|207|
| jaguar|      sedan|262|
|porsche|convertible|207|
+-------+-----------+---+

+---+------+------+-------------+------+
|age|deptid|gender|         name|salary|
+---+------+------+-------------+------+
| 40|   200|  male|Holland Drake|  4500|
| 51|   100|female|June Rutledge|  4300|
| 44|   200|  male|Nielsen Knapp|  6500|
+---+------+------+-------------+------+



#pandas data frame is stored in the master node


In [59]:
#pandas data frame is stored in the master node
empPands = empDf.toPandas()
for index, row in empPands.iterrows():
    print(row["salary"])

3000
4500
2700
4300
6500
